In [1]:
import numpy as np
import pandas as pd
import cvxopt
from cvxopt import matrix, solvers
import math

In [2]:
train_data = pd.read_csv("spam_train.data",header = None)
test_data = pd.read_csv("spam_test.data",header = None)
Valid_data = pd.read_csv("spam_validation.data",header = None)

In [3]:
def targetvar(data):
    data.loc[data[57]==0,57]=-1
    return data

In [4]:
train_data = targetvar(train_data)
test_data = targetvar(test_data)
Valid_data = targetvar(Valid_data)

In [5]:
def split(data):
    n = len(data.columns)-2
    X = data.iloc[:,0:n+1]
    Y = data.iloc[:,[n+1]]
    return (X,Y)

In [6]:
train_x,train_y = split(train_data)
Valid_x,Valid_y = split(Valid_data)
test_x,test_y = split(test_data)

In [7]:
m,n = train_x.shape[0],train_x.shape[1]

In [8]:
# def Gaussian(x1,x2,sigma):
#     k = -1*sum((x1-x2)**2)/(2*sigma)
#     return math.exp(k)

In [9]:
def Gaussian(x1,x2,sigma):
    K = np.zeros((x1.shape[0], x2.shape[0]))
    for i, x in enumerate(x1):
        for j, y in enumerate(x2):
            K[i, j] = np.exp(-(1/2*sigma) * sum((x - y) ** 2))
    return K

In [10]:
def P_calc(sigma):
    y_val = train_y[57].to_numpy()
    y = y_val.reshape(1, -1)
    x = train_x.to_numpy()
#     for i in range(m):
#         for j in range(m):
#             P[i][j]=Y[i]*Y[j]*Gaussian(X[i],X[j],sigma)
    calc = np.dot(y.T, y) * Gaussian(x, x, sigma)
    return matrix(calc,tc='d')

In [11]:
def Q_calc():
    Q = -1*np.ones((m,1))
    return matrix(Q,tc='d')

In [12]:
def G_calc():
    x1 = -1*np.eye(m)
    x2 = np.eye(m)
    G = np.append(x1,x2,axis=0)
    return matrix(G, tc='d')

In [13]:
def H_calc(C):
    x1 = np.zeros((m,1))
    x2 = C*np.ones((m,1))
    H = np.append(x1,x2,axis=0)
    return matrix(H,tc='d')

In [14]:
def A_calc(Y):
    return matrix(Y,tc='d')

In [15]:
def B_calc():
    return matrix(np.zeros((1,1)),tc='d')

In [16]:
def Accuracy(X,Y,W,b):
    prod =X.dot(np.array(W))+b
    Y = Y[57].to_numpy()
    print(prod.shape)
    print(Y.shape)
    res = sum(Y*np.array(prod)>0)
    return (res/len(X)*100)

In [17]:
# def Bias(X,Y,W):
#     b = np.mean(Y-np.dot(X,W))

In [18]:
Q,G,A,B = Q_calc(),G_calc(),A_calc(train_y[57].to_numpy()).T,B_calc()

In [19]:
Valid_acc =[0]*45
train_acc = [0]*45
test_acc = 0
final_C = 0
final_b=0
final_w = []
final_sigma = 0
W_list=[]
b_list=[]

In [20]:
X = train_x.to_numpy()
Y = train_y[57].to_numpy()

In [ ]:
for i in range(1):
    for l in range(1):
        C = 10**i
        sigma = 10**l
        P = P_calc(sigma)
        H = H_calc(C)
        result = cvxopt.solvers.qp(P, Q, G, H,A,B)
        Thres = 1e-05
        al = np.ravel(result['x'])
#         alphas = np.where(al>Thres,al,0)
        S = (al > Thres).reshape(-1, )
#         temp = Y*alphas
#         W = sum(X*temp[:, np.newaxis])
        W=np.dot(X.T,al*Y)
        b = Y[S] - np.dot(X[S], W) # b calculation
        b = np.mean(b)
#         b = np.mean(Y-np.dot(X,W))
        W_list.append(W)
        b_list.append(b)
        train_Acc = Accuracy(train_x,train_y,W,b)
        train_acc[i]= train_Acc
        print("Training accuracy with C:",C," and sigma:",sigma," is", train_Acc)
        Valid_Acc = Accuracy(Valid_x,Valid_y,W,b)
        print("Validation accuracy with C:",C," and sigma:",sigma," is", Valid_Acc)
        test_Acc = Accuracy(test_x,test_y,W,b)
        print("Test accuracy with C:",C," and sigma:",sigma," is", test_Acc)
        if Valid_Acc>max(Valid_acc):
            final_C = C
            final_b = b 
            final_w = W
            final_sigma = sigma
        Valid_acc[i] = Valid_Acc

In [22]:
W_list

[array([ 8.52831462e+01,  9.64290638e+01,  1.80861234e+02,  2.03664761e+02,
         3.22135231e+02,  1.40040910e+02,  3.06631092e+02,  1.95066746e+02,
         1.20649068e+02,  2.04373897e+02,  1.15380655e+02, -1.17513176e+02,
         8.92207364e+01,  4.13651426e+01,  9.01024752e+01,  5.44739736e+02,
         2.58247660e+02,  2.34841420e+02,  1.31134523e+03,  2.19499305e+02,
         9.69858246e+02,  3.20101503e+02,  2.56664380e+02,  2.07544490e+02,
        -1.35116213e+03, -6.16023311e+02, -8.03317919e+02, -2.63130063e+02,
        -2.56000945e+02, -2.15857570e+02, -1.40268792e+02, -8.32085296e+01,
        -1.66510773e+02, -8.28958095e+01, -2.29171099e+02, -1.67615281e+02,
        -2.27479127e+02, -1.51441641e+00, -1.60526142e+02, -4.81739943e+01,
        -6.42420717e+01, -3.37871223e+02, -1.02087684e+02, -1.55965387e+02,
        -2.82420773e+02, -7.77454619e+01, -6.36945986e+00, -6.18172379e+01,
        -1.63260885e+01, -6.95659196e+01, -1.78521093e+01,  4.73706002e+02,
         1.7

In [ ]:
b_list

In [ ]:
test_acc = Accuracy(test_x,test_y,final_w,final_b)